<a href="https://colab.research.google.com/github/unicamp-dl/IA025_2022S1/blob/main/ex08/patrick_ferreira/ex08_patrick_ferreira.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
nome = "Patrick de Carvalho Tavares Rezende Ferreira"
print(f'Meu nome é {nome}')

Meu nome é Patrick de Carvalho Tavares Rezende Ferreira


#  Exercício: Modelo de Linguagem com auto-atenção

Este exercício é similar ao da Aula 7, mas iremos agora treinar uma rede neural *com auto-atenção* para prever a próxima palavra de um texto, data as palavras anteriores como entrada. 

Na camada de auto-atenção, não se esqueça de implementar:
- Embeddings de posição
- Projeções lineares (WQ, WK, WV, WO)
- Conexões residuais
- Camada de feed forward (2-layer MLP)



O dataset usado neste exercício (BrWaC) possui um tamanho razoável e você vai precisar rodar seus experimentos com GPU.

Alguns conselhos úteis:
- **ATENÇÃO:** o dataset é bem grande. Não dê comando de imprimí-lo.
- Durante a depuração, faça seu dataset ficar bem pequeno, para que a depuração seja mais rápida e não precise de GPU. Somente ligue a GPU quando o seu laço de treinamento já está funcionando
- Não deixe para fazer esse exercício na véspera. Ele é trabalhoso.

In [2]:
# iremos utilizar a biblioteca dos transformers para ter acesso ao tokenizador do BERT.
!pip install transformers

## Importação dos pacotes

In [3]:
import collections
import itertools
import functools
import math
import random

import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader
from tqdm import tqdm_notebook


In [4]:
# Check which GPU we are using
!nvidia-smi

Wed May 25 18:34:51 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.73.05    Driver Version: 510.73.05    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0  On |                  N/A |
| 65%   47C    P8    13W / 170W |    387MiB / 12288MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
if torch.cuda.is_available():
    dev = "cuda:0"
else:
    dev = "cpu"
device = torch.device(dev)
print('Using {}'.format(device))

Using cuda:0


## Implementação do MyDataset

In [6]:
from tqdm import tqdm
from typing import List


def tokenize(text: str, tokenizer):
    return tokenizer(text, return_tensors=None, add_special_tokens=False).input_ids


class MyDataset():
    def __init__(self, texts: List[str], tokenizer, context_size: int):
        # Escreva seu código aqui
        try:
            self.x = np.load("x_" + str(len(texts)) + ".npy", mmap_mode="r", allow_pickle=True)
            self.y = np.load("y_" + str(len(texts)) + ".npy", mmap_mode="r", allow_pickle=True)

            print("Carregando dataset preprocessado")

        except Exception as e:
            print("Excecao: ", e)
            print("Montando dataset")

            self.x = list()
            self.y = list()

            for text in tqdm(texts):
                tokens_key = tokenize(text, tokenizer)
                for i in range(len(tokens_key)-context_size):
                    self.x.append(tokens_key[i:i+context_size])
                    self.y.append(tokens_key[i+context_size])

            self.x = np.array(self.x)
            self.y = np.array(self.y)

            np.save("x_" + str(len(texts)) + ".npy", self.x)
            np.save("y_" + str(len(texts)) + ".npy", self.y)

    def __len__(self):
        # Escreva seu código aqui
        return len(self.y)

    def __getitem__(self, idx):
        # Escreva seu código aqui
        return torch.tensor(self.x[idx]).long(), torch.tensor(self.y[idx]).long()

## Testando se a implementação do MyDataset está correta

In [7]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("neuralmind/bert-base-portuguese-cased")

dummy_texts = ['Eu gosto de correr', 'Ela gosta muito de comer pizza']

dummy_dataset = MyDataset(texts=dummy_texts, tokenizer=tokenizer, context_size=3)
dummy_loader = DataLoader(dummy_dataset, batch_size=6, shuffle=False)
assert len(dummy_dataset) == 5
print('passou no assert de tamanho do dataset')

first_batch_input, first_batch_target = next(iter(dummy_loader))

correct_first_batch_input = torch.LongTensor(
    [[3396, 10303, 125],
     [1660, 5971, 785],
     [5971, 785, 125],
     [785, 125, 1847],
     [125, 1847, 13779]])

correct_first_batch_target = torch.LongTensor([13239, 125, 1847, 13779, 15616])

assert torch.equal(first_batch_input, correct_first_batch_input)
print('Passou no assert de input')
assert torch.equal(first_batch_target, correct_first_batch_target)
print('Passou no assert de target')

Carregando dataset preprocessado
passou no assert de tamanho do dataset
Passou no assert de input
Passou no assert de target


# Carregamento do dataset 

Iremos usar uma pequena amostra do dataset [BrWaC](https://www.inf.ufrgs.br/pln/wiki/index.php?title=BrWaC) para treinar e avaliar nosso modelo de linguagem.

In [8]:
!wget -nc https://storage.googleapis.com/unicamp-dl/ia025a_2022s1/aula7/sample_brwac.txt

File ‘sample_brwac.txt’ already there; not retrieving.



In [9]:
# Load datasets
context_size = 9

valid_examples = 101
test_examples = 100
texts = open('sample_brwac.txt').readlines()

# print('Truncating for debugging purposes.')
# texts = texts[:500]

training_texts = texts[:-(valid_examples + test_examples)]
valid_texts = texts[-(valid_examples + test_examples):-test_examples]
test_texts = texts[-test_examples:]

training_dataset = MyDataset(texts=training_texts, tokenizer=tokenizer, context_size=context_size)
valid_dataset = MyDataset(texts=valid_texts, tokenizer=tokenizer, context_size=context_size)
test_dataset = MyDataset(texts=test_texts, tokenizer=tokenizer, context_size=context_size)

Carregando dataset preprocessado
Carregando dataset preprocessado
Carregando dataset preprocessado


In [10]:
print(f'training examples: {len(training_dataset)}')
print(f'valid examples: {len(valid_dataset)}')
print(f'test examples: {len(test_dataset)}')

training examples: 27675765
valid examples: 82250
test examples: 166726


In [11]:
class LanguageModel(torch.nn.Module):

    def __init__(self, vocab_size, context_size, embedding_dim):
        """
        Implements the Self-attention, decoder-only."

        Args:
            vocab_size (int): Size of the input vocabulary.
            context_size (int): Size of the sequence to consider as context for prediction.
            embedding_dim (int): Dimension of the embedding layer for each word in the context.
        """
        super().__init__()

        self.context_size = context_size
        self.embedding_dim = embedding_dim

        # hidden_size for MLP
        hidden_size = 10 * embedding_dim

        # tokens (words indexes) embedding and positional embedding
        self.c_embedding = nn.Embedding(vocab_size, embedding_dim)
        self.p_embedding = nn.Embedding(vocab_size, embedding_dim)

        # Linear projections
        self.w_q = nn.Sequential(
            nn.Linear(embedding_dim, hidden_size),
            nn.Tanh(),
            nn.Linear(hidden_size, embedding_dim)
        )
        self.w_k = nn.Sequential(
            nn.Linear(embedding_dim, hidden_size),
            nn.Tanh(),
            nn.Linear(hidden_size, embedding_dim)
        )
        self.w_v = nn.Sequential(
            nn.Linear(embedding_dim, hidden_size),
            nn.Tanh(),
            nn.Linear(hidden_size, embedding_dim)
        )
        self.w_0 = nn.Sequential(
            nn.Linear(embedding_dim, hidden_size),
            nn.Tanh(),
            nn.Linear(hidden_size, embedding_dim)
        )

        # output MLP
        self.mlp = nn.Sequential(
            nn.Linear(embedding_dim, hidden_size),
            nn.Tanh(),
            nn.Linear(hidden_size, embedding_dim)
        )

        self.linear_output = nn.Linear(embedding_dim, vocab_size)

        # cast to probabilities
        self.softmax = nn.Softmax(dim=1)

        self.skip_connection1 = nn.Sequential(
            nn.Linear(embedding_dim, embedding_dim)
        )

        self.skip_connection2 = nn.Sequential(
            nn.Linear(embedding_dim, embedding_dim)
        )

        self.norm1 = nn.LayerNorm(embedding_dim)
        self.norm2 = nn.LayerNorm(embedding_dim)

    def forward(self, inputs):
        """
        Args:
            inputs is a LongTensor of shape (batch_size, context_size)

        Returns:
            logits of shape (batch_size, vocab_size)
        """
        # Escreva seu código aqui.

        input_embeddings = self.c_embedding(inputs)

        positional_indexes = torch.arange(self.context_size).view(1, -1).repeat(inputs.shape[0], 1).to(device)
        positional_embeddings = self.p_embedding(positional_indexes)

        x_embeddings = positional_embeddings + input_embeddings

        k = self.w_k(x_embeddings)
        v = self.w_v(x_embeddings)
        q = self.w_q(x_embeddings)

        scores = torch.matmul(q, k.transpose(1, 2))
        probabilities = self.softmax(scores)

        e = self.w_0(self.norm1(self.skip_connection1(x_embeddings) + torch.matmul(probabilities, v)))

        # We only take the last word to make prediction
        logits = self.mlp(e[:, -1, :].flatten(start_dim=1))

        return self.linear_output(self.norm2(logits + self.skip_connection2(e[:, -1, :].flatten(start_dim=1))))

## Teste o modelo com um exemplo

In [12]:
model = LanguageModel(
    vocab_size=tokenizer.vocab_size,
    context_size=context_size,
    embedding_dim=64,
).to(device)

sample_train, _ = next(iter(DataLoader(training_dataset)))
sample_train_gpu = sample_train.to(device)
model(sample_train_gpu).shape

torch.Size([1, 29794])

In [13]:
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Number of model parameters: {num_params}')

Number of model parameters: 6171938


## Assert da Perplexidade


In [14]:
random.seed(123)
np.random.seed(123)
torch.manual_seed(123)


def perplexity(logits, target):
    """
    Computes the perplexity.

    Args:
        logits: a FloatTensor of shape (batch_size, vocab_size)
        target: a LongTensor of shape (batch_size,)

    Returns:
        A float corresponding to the perplexity
    """
    loss = nn.functional.cross_entropy(logits, target, reduction='mean')
    return torch.exp(loss)


# n_examples = 1000
#
# sample_train, target_token_ids = next(iter(DataLoader(training_dataset, batch_size=n_examples)))
# sample_train_gpu = sample_train.to(device)
# target_token_ids = target_token_ids.to(device)
# logits = model(sample_train_gpu)
#
# my_perplexity = perplexity(logits=logits, target=target_token_ids)
#
# print(f'my perplexity:              {int(my_perplexity)}')
# print(f'correct initial perplexity: {tokenizer.vocab_size}')
#
# assert math.isclose(my_perplexity, tokenizer.vocab_size, abs_tol=7000)
# print('Passou o no assert da perplexidade')

## Laço de Treinamento e Validação

In [15]:
from queue import Queue
from threading import Thread


class DataManager(Thread):
    def __init__(self, data_loader, buffer_size=3, device=torch.device("cpu"), data_type=torch.float32):
        """
This manager intends to load a PyTorch dataloader like from disk into memory,
reducing the acess time. It does not easily overflow memory, because we set a
buffer size limiting how many samples will be loaded at once. Everytime a sample
is consumed by the calling thread, another one is replaced in the
buffer (unless we reach the end of dataloader).

A manger may be called exactly like a dataloader, an it's based in an internal
thread that loads samples into memory in parallel. This is specially useful
when you are training in GPU and processor is almost idle.

        :param data_loader: Base dataloader to load in parallel.
        :param buffer_size: How many samples to keep loaded (caution to not overflow RAM). Default: 3.
        :param device: Torch device to put samples in, like torch.device("cpu") (default). It saves time by transfering in parallel.
        :param data_type: Automatically casts tensor type. Default: torch.float32.
        """
        super().__init__()
        self.buffer_queue = Queue(maxsize=buffer_size)
        self.data_loader = data_loader
        self.buffer_size = buffer_size
        self.device = device
        self.data_type = data_type

        self.dataloader_finished = False

    def run(self):
        """
Runs the internal thread that iterates over the dataloader until fulfilling the
buffer or the end of samples.
        """
        for i, (x, y) in enumerate(self.data_loader):
            # Important to set before put in queue to avoid race condition
            # would happen if trying to get() in next() method before setting this flag
            if i >= len(self) - 1:
                self.dataloader_finished = True

            self.buffer_queue.put([x.to(self.data_type).to(self.device),
                                   y.to(self.data_type).to(self.device)])

    def __iter__(self):
        """
Returns an iterable of itself.

        :return: Iterable around this class.
        """
        self.start()
        self.dataloader_finished = False
        return self

    def __next__(self):
        """
Intended to be used as iterator.

        :return: Next iteration element.
        """
        if self.dataloader_finished is True and self.buffer_queue.empty():
            raise StopIteration()

        return self.buffer_queue.get()

    def __len__(self):
        return len(self.data_loader)

In [16]:
max_examples = 800_000_000
eval_every_steps = 10000
lr = 3e-4

model = LanguageModel(
    vocab_size=tokenizer.vocab_size,
    context_size=context_size,
    embedding_dim=128,
).to(device)

train_loader = DataLoader(training_dataset, batch_size=1024, shuffle=True, drop_last=True, num_workers=2)
validation_loader = DataLoader(valid_dataset, batch_size=1024, num_workers=2)

optimizer = torch.optim.Adam(model.parameters(), lr=lr)


def train_step(input, target):
    model.train()
    model.zero_grad()

    logits = model(input.to(device))
    loss = nn.functional.cross_entropy(logits, target.to(device))
    loss.backward()
    optimizer.step()

    return loss.item()


def validation_step(input, target):
    model.eval()
    logits = model(input)
    loss = nn.functional.cross_entropy(logits, target)
    return loss.item()


train_losses = []
n_examples = 0
step = 0
while n_examples < max_examples:
    for input, target in DataManager(train_loader, device=device, buffer_size=4, data_type=None):
        loss = train_step(input.to(device), target.to(device))
        train_losses.append(loss)

        if step % eval_every_steps == 0:
            train_ppl = np.exp(np.average(train_losses))

            with torch.no_grad():
                valid_ppl = np.exp(np.average([
                    validation_step(input.to(device), target.to(device))
                    for input, target in DataManager(validation_loader, device=device, buffer_size=4, data_type=None)]))

            print(f'{step} steps; {n_examples} examples so far; train ppl: {train_ppl:.2f}, valid ppl: {valid_ppl:.2f}')
            train_losses = []

        n_examples += len(input)  # Increment of batch size
        step += 1
        if n_examples >= max_examples:
            break

0 steps; 0 examples so far; train ppl: 34538.93, valid ppl: 25952.82
10000 steps; 10240000 examples so far; train ppl: 529.76, valid ppl: 337.01
20000 steps; 20480000 examples so far; train ppl: 307.70, valid ppl: 280.29
30000 steps; 30720000 examples so far; train ppl: 272.73, valid ppl: 262.06
40000 steps; 40960000 examples so far; train ppl: 256.16, valid ppl: 252.97
50000 steps; 51200000 examples so far; train ppl: 249.18, valid ppl: 245.87
60000 steps; 61440000 examples so far; train ppl: 316.18, valid ppl: 248.65
70000 steps; 71680000 examples so far; train ppl: 218.31, valid ppl: 203.66
80000 steps; 81920000 examples so far; train ppl: 190.76, valid ppl: 184.61
90000 steps; 92160000 examples so far; train ppl: 171.85, valid ppl: 173.23
100000 steps; 102400000 examples so far; train ppl: 163.58, valid ppl: 164.38
110000 steps; 112640000 examples so far; train ppl: 156.44, valid ppl: 160.74
120000 steps; 122880000 examples so far; train ppl: 151.53, valid ppl: 154.67
130000 steps;

KeyboardInterrupt: 

## Avaliação final no dataset de teste


Bonus: o modelo com menor perplexidade no dataset de testes ganhará 0.5 ponto na nota final.

In [17]:
test_loader = DataLoader(test_dataset, batch_size=64)

with torch.no_grad():
    test_ppl = np.exp(np.average([
        validation_step(input.to(device), target.to(device))
        for input, target in test_loader
    ]))

print(f'test perplexity: {test_ppl}')

test perplexity: 109.97276644260708


## Teste seu modelo com uma sentença

Escolha uma sentença gerada pelo modelo que ache interessante.

In [18]:
prompt = 'Eu gosto de comer pizza pois me faz'
max_output_tokens = 20
model.eval()

for _ in range(max_output_tokens):
    input_ids = tokenize(text=prompt, tokenizer=tokenizer)
    input_ids_truncated = input_ids[-context_size:]  # Usamos apenas os últimos <context_size> tokens como entrada para o modelo.
    logits = model(torch.LongTensor([input_ids_truncated]).to(device))
    # Ao usarmos o argmax, a saída do modelo em cada passo é o token de maior probabilidade.
    # Isso se chama decodificação gulosa (greedy decoding).
    predicted_id = torch.argmax(logits).item()
    input_ids += [predicted_id]  # Concatenamos a entrada com o token escolhido nesse passo.
    prompt = tokenizer.decode(input_ids)
    print(prompt)

Eu gosto de comer pizza pois me faz muito
Eu gosto de comer pizza pois me faz muito bem
Eu gosto de comer pizza pois me faz muito bem,
Eu gosto de comer pizza pois me faz muito bem, mas
Eu gosto de comer pizza pois me faz muito bem, mas não
Eu gosto de comer pizza pois me faz muito bem, mas não é
Eu gosto de comer pizza pois me faz muito bem, mas não é um
Eu gosto de comer pizza pois me faz muito bem, mas não é um bom
Eu gosto de comer pizza pois me faz muito bem, mas não é um bom tempo
Eu gosto de comer pizza pois me faz muito bem, mas não é um bom tempo,
Eu gosto de comer pizza pois me faz muito bem, mas não é um bom tempo, mas
Eu gosto de comer pizza pois me faz muito bem, mas não é um bom tempo, mas não
Eu gosto de comer pizza pois me faz muito bem, mas não é um bom tempo, mas não é
Eu gosto de comer pizza pois me faz muito bem, mas não é um bom tempo, mas não é um
Eu gosto de comer pizza pois me faz muito bem, mas não é um bom tempo, mas não é um bom
Eu gosto de comer pizza pois m